<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_crewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet

modelid="llama3.1"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
from google.colab import userdata

ngrok.kill()
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="11434", proto="http", bind_tls=True, host_header="localhost:11434" )
print("OLLama URL:", ngrok_tunnel.public_url)

llama_url = ngrok_tunnel.public_url


## CrewAI - simple agent


In [2]:
!pip install crewai crewai-tools --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.3/537.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB

In [ ]:
# src/latest_ai_development/crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool

@CrewBase
class  MyTestCrew():
  """ Crew to test CrewAI"""

  @agent
  def researcher(self) -> Agent:
     return Agent (
         role="{topic} Senior Data Researcher",
         goal="Uncover cutting-edge developments in {topic}",
         backstory="You're a researcher for latest development in {topic}. You are presenting information in a clear and consise manner.",
         verbose=True
     )

  @agent
  def meeting_scheduler(self) -> Agent:
    return Agent (
         role="{topic} Senior Project Manager",
         goal="Create a list of topic points for a meeting to discuss {topic}",
         backstory="You're a Project Manager who is focussed on the most relevant details. Your are presenting the topics in a clear and short manner to your team members",
         verbose=True
    )

  @task
  def research_task(self) -> Task:
    return Task(
        agent="researcher",
        description="Conduct a thorough research about {topic}",
        expected_output="A list with 10 bullet points of the most relevant information about {topic}",
    )

  @task
  def reporting_task(self) -> Task:
    return Task(
      agent="meeting_scheduler",
      description="Review the context you got and expand each topic into a full section for a report. Make sure the report is detailed and contains any and all relevant information.",
      expected_output="A fully fledge reports with the mains topics. Formatted as markdown without '```'",
      output_file='output/report.md' # This is the file that will be contain the final report.
    )

  @crew
  def crew(self) ->Crew:
      """ detects methode that have the @agent and @task decorator
        and creates a crew object with all the agents and tasks
      """
      return Crew(
          agents=self.agents, # automatic detects factory methods with @agent decorator
          tasks=self.taks, # automatic detects factory methods with @task decorator
          process = Process.sequential,
          verbose=True,
      )